# CliqueSNV validation protocols

In [2]:
from pathlib import Path
import os
import subprocess
import pandas as pd
import glob
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import re
import json

In [3]:
#Haplotyping tools and configs:
CliqueSNV_fn = "/home/code/cliquesnv/1.5.10/clique-snv.jar"
aBayesQR_fn = "/home/code/abayesqr/aBayesQR/aBayesQR"
PredictHaplo_fn = "/home/code/PredictHaplo-Paired-0.4/PredictHaplo-Paired"

In [4]:
#other constants:
samtools_dn = "/home/code/simseq/SimSeq"
picard_dn = "/home/code/picard/picard-tools-1.119"

base_dn = "/alina-data0/sergey/CliqueSNV"

HXB2_pol_ref_fn = "refs/HXB2_pol_ref/ref.fas"
IAV_ref_fn ="refs/IAV_ref/ref.fa"

subsample_fn = "scripts/Subsampler.py"
sim_read_generator_fn = "scripts/ReadGenerator.py"

# 1) Data preparation

In [5]:
experimental_datasets = ["HIV9exp", "HIV2exp"]
reduced_experimental_datasets = ["{}_50k_reads".format(x) for x in experimental_datasets]
labmix_dataset = ["HIV5exp"]
simulated_datasets = ["HIV7sim", "IAV10sim"]
er1 = ["reads/{}_R1.fastq.gz".format(x) for x in experimental_datasets]
er2 = ["reads/{}_R2.fastq.gz".format(x) for x in experimental_datasets]
rer1 = ["reads/{}_R1.fastq.gz".format(x) for x in reduced_experimental_datasets]
rer2 = ["reads/{}_R2.fastq.gz".format(x) for x in reduced_experimental_datasets]
labmix1 = ["reads/SRR961514_1.fastq"]
labmix2 = ["reads/SRR961514_1.fastq"]
sim_relevant_hapl = ["relevant_haplotypes/HIV7sim.fasta", "relevant_haplotypes/IAV10sim.fasta"]
rs1 = ["reads/{}_R1.fastq.gz".format(x) for x in simulated_datasets]
rs2 = ["reads/{}_R2.fastq.gz".format(x) for x in simulated_datasets]
exp_sams = ["alignment/{}.sam".format(x) for x in experimental_datasets]
reduced_exp_sams = ["alignment/{}.sam".format(x) for x in reduced_experimental_datasets]
labmix_sam = ["alignment/{}.sam".format(x) for x in labmix_dataset]
simulated_sams = ["alignment/{}.sam".format(x) for x in simulated_datasets]

## Prepare simulated HIV7sim and IAV10sim datasets

In [22]:
%%capture
for i in range(len(rs1)):
    op = Path("reads/tmp")
    op.mkdir(parents=True, exist_ok=True)
    od = str(op)
    iref = sim_relevant_hapl[i]
    !python3 $sim_read_generator_fn -c 50000 -s $samtools_dn -p $picard_dn -i $iref -o $od
    
    i1 = Path(od, "sim_reads.1.fastq")
    i2 = Path(od, "sim_reads.2.fastq")
    o1 = rs1[i]
    o2 = rs2[i]
    
    !gzip -c $i1 > $o1
    !gzip -c $i2 > $o2
    !rm -rf $od

## Reduced HIV9exp and HIV2exp so that the datasets to consist of just 50k reads

In [21]:
for i in range(len(er1)):
    i1 = er1[i]
    i2 = er2[i]
    ir1 = rer1[i]
    ir2 = rer2[i]
    !python3 $subsample_fn --n-samples 50000 --fastq1 $i1 --fastq2 $i2 --fastq1_out $ir1 --fastq2_out $ir2

## Aligning reads

In [19]:
%%capture
!bwa index $HXB2_pol_ref_fn
!bwa index $IAV_ref_fn
all_r1 = er1 + rer1 + labmix1 + rs1
all_r2 = er2 + rer2 + labmix2 + rs2
all_sams = exp_sams + reduced_exp_sams + labmix_sam + simulated_sams
refs = [HXB2_pol_ref_fn] * 8 + [IAV_ref_fn]
for i in range(len(all_r1)):
    i1 = all_r1[i]
    i2 = all_r2[i]
    o = all_sams[i]
    r = refs[i]
    !bwa mem -B 2 $r $i1 $i2 > $o

# 2) Protocol of Sensitivity and Specificity analysis for CliqueSNV

## Running CliqueSNV on HIV9exp, HIV2exp, and HIV5exp with different sensitivity thresholds

In [83]:
%%capture
datasets = experimental_datasets \
    + reduced_experimental_datasets \
    + labmix_dataset \
    + simulated_datasets
for d in datasets:
    for x in [0.2, 0.1, 0.05, 0.02, 0.01, 0.005, 0.002, 0.001]:
        out_dir = str(Path(base_dn, "results", "{}_{}p_CliqueSNV".format(d, x*100)))
        in_f = str(Path(base_dn, "alignment", "{}.sam".format(d)))
        !java -Xmx100g -jar $CliqueSNV_fn -m snv-illumina -tf $x -outDir $out_dir -in $in_f

In [25]:
%%capture
datasets = experimental_datasets \
    + reduced_experimental_datasets \
    + labmix_dataset \
    + simulated_datasets
for d in datasets:
    for x in [10, 15, 20, 30, 50, 70, 100, 150, 200, 300, 500]:
        out_dir = str(Path(base_dn, "results", "{}_2p_{}t_CliqueSNV".format(d, x)))
        in_f = str(Path(base_dn, "alignment", "{}.sam".format(d)))
        !java -Xmx100g -jar $CliqueSNV_fn -m snv-illumina -tf 0.02 -t $x -outDir $out_dir -in $in_f

# 3) Protocol of comparison of CliqueSNV with Consensus, PredictHaplo, aBayesQR, and 2SNV

### PredictHaplo
PredictHaplo failed on HIV7sim and HIV9exp.
HIV9exp region reduced from 1:1074 to 1:1065.
HIV7sim region reduced from 1:1074 to 1:1055.

In [22]:
%%capture
datasets = experimental_datasets \
    + reduced_experimental_datasets \
    + labmix_dataset \
    + simulated_datasets
for d in datasets:
    config_fn = str(Path(base_dn,"tool_configs/{}_PredictHaplo.config".format(d)))
    out_dir = str(Path(base_dn,"results/{}_PredictHaplo".format(d)))
    Path(out_dir).mkdir(parents=True, exist_ok=True)
    os.chdir(out_dir)
    !$PredictHaplo_fn $config_fn
    os.chdir(base_dn)

### aBayesQR
aBayesQR didn't finish on a full HIV2exp and HIV9exp datasets

In [150]:
%%capture
datasets = reduced_experimental_datasets \
    + labmix_dataset \
    + simulated_datasets
for d in datasets:
    config_fn = str(Path(base_dn,"tool_configs/{}_aBayesQR.config".format(d)))
    out_dir = str(Path(base_dn,"results/{}_aBayesQR".format(d)))
    Path(out_dir).mkdir(parents=True, exist_ok=True)
    os.chdir(out_dir)
    !$aBayesQR_fn $config_fn
    os.chdir(base_dn)

### Consensus

In [7]:
%%capture
datasets = experimental_datasets \
    + reduced_experimental_datasets \
    + labmix_dataset \
    + simulated_datasets
for d in datasets:
    out_dir = str(Path(base_dn, "results/{}_consensus".format(d)))
    in_f = str(Path(base_dn,"alignment/{}.sam".format(d)))
    !java -jar $CliqueSNV_fn -m consensus-illumina -in $in_f -outDir $out_dir

# Create standard fasta for tools' results

In [7]:
regions = {"HIV9exp":(0,1065),
           "HIV2exp":(4,1074),
           "HIV5exp":(0,1074),
           "HIV7sim":(24,1050),
           "IAV10sim":(0,2263)
          }
a=list(map(lambda x: [x[0],x[1][0],x[1][1]],regions.items()))
b=[[r[col] for r in a] for col in range(len(a[0]))]
datasets = pd.DataFrame({"dataset":b[0], "begin_pos":b[1], "end_pos":b[2]})
tools = pd.DataFrame({"tool":["aBayesQR", "PredictHaplo", "CliqueSNV", "consensus"]})
datasets["key"] = 0
tools["key"] = 0

results = pd.merge(datasets, tools, on="key")
results = results.drop(columns=["key"])
column_names=["dataset", "tool", "begin_pos", "end_pos"]
results = results.reindex(columns=column_names)

In [8]:
tool_results_paths = list()
results_paths = list()
for i,row in results.iterrows():
    n = [row["dataset"]]
    if row["tool"] == "aBayesQR" and (row["dataset"] == "HIV9exp" or
                                      row["dataset"] == "HIV2exp"):
        n.append("50k_reads")
    elif row["tool"] == "CliqueSNV":
        n.append("2.0p")
    n.append(row["tool"])
    tool_results_paths.append("results/"+"_".join(n))
    results_paths.append("results/"+"_".join([row["dataset"], row["tool"]])+'.fasta')
results["tool_out_path"] = tool_results_paths
results["output_fasta"] = results_paths

In [9]:
def standard_fasta_aBayesQR(indir, outf, begin_pos, end_pos):
    ins=list()
    with open(indir+"/test_Seq.txt") as f:
        for s in f:
            ins.append(s.strip()[begin_pos:end_pos])
    with open(indir+"/test_Freq.txt") as f:
        infr=next(f).strip().split()
    seqs=list()
    for i,s in enumerate(ins):
        sn="{}_{}".format(str(i),infr[i])
        seqs.append(SeqRecord(Seq(s),id=sn,description=sn))
    SeqIO.write(seqs,outf,"fasta")
    
def standard_fasta_PredictHaplo(indir, outf, begin_pos, end_pos):
    fs=re.findall("[^,]*ph_global[^,]*\.fas",",".join(glob.glob(indir+"/*")))
    seq_beg_pos=0
    seq_end_pos=0
    best_range=0
    for fn in fs:
        a=fn.split(".")[0].split("_")
        b,e=int(a[-2]),int(a[-1])
        if e-b>best_range:
            best_range=e-b
            seq_beg_pos=b-1
            seq_end_pos=e
    new_beg_pos = max(0,begin_pos-seq_beg_pos)
    new_end_pos = min(seq_end_pos-seq_beg_pos, end_pos-seq_beg_pos)
    fn = indir + "/ph_global_{}_{}.fas".format(str(seq_beg_pos+1), str(seq_end_pos))
    with open(fn) as f:
        r="".join(f.read()).replace("\n","")
    seqs=list()
    hs=re.findall(">[^>]*;Freq:(\d+\.\d+)[^>]*;EndOfComments([^>]*)",r)
    for i,h in enumerate(hs):
        n="{}_{}".format(i,h[0])
        seqs.append(SeqRecord(Seq(h[1][new_beg_pos:new_end_pos]),id=n,description=n))
    SeqIO.write(seqs,outf,"fasta")
    
def standard_fasta_CliqueSNV(indir, outf, begin_pos, end_pos):
    fi=glob.glob(indir+"/*.fasta")[0]
    seqs=list(SeqIO.parse(fi,"fasta"))
    for s in seqs:
        s.seq = Seq(str(s.seq)[begin_pos:end_pos])
    SeqIO.write(seqs, outf, "fasta")
    
def standard_fasta_consensus(indir, outf, begin_pos, end_pos):
    fi=glob.glob(indir+"/*.fasta")[0]
    seqs=list(SeqIO.parse(fi,"fasta"))
    seqs[0].id = "0_fr_1.0"
    seqs[0].seq = Seq(str(seqs[0].seq)[begin_pos:end_pos])
    SeqIO.write(seqs, outf, "fasta")

for i,row in results.iterrows():
    globals()["standard_fasta_"+row["tool"]](row["tool_out_path"], row["output_fasta"], row["begin_pos"], row["end_pos"])

## Creating fasta with relevant haplotypes

In [10]:
for d in results.dataset.unique():
    begin_pos=results[results.dataset==d].begin_pos.min()
    end_pos=results[results.dataset==d].end_pos.max()
    seqs=list(SeqIO.parse("relevant_haplotypes/"+d+".fasta", "fasta"))
    for s in seqs:
        s.seq=s[begin_pos:end_pos].seq
    SeqIO.write(seqs, "relevant_haplotypes/"+d+"_trm"+".fasta", 'fasta')

In [11]:
relevant_haplotypes=list()
for i,row in results.iterrows():
    n = row["dataset"]
    relevant_haplotypes.append("relevant_haplotypes/{}_trm.fasta".format(n))
results["relevant_haplotypes_fasta"] = relevant_haplotypes

## Collect statistics

In [12]:
stat = dict()
for i,row in results.iterrows():
    n = "_".join([row["dataset"], row["tool"]])
    p = row["output_fasta"]
    r = row["relevant_haplotypes_fasta"]
    a = !python scripts/analyze_prediction.py $p $r
    stat[n] = json.loads(a[0])

In [13]:
stat_keys = ["EMD","TP","FP","Et->p","Et<-p"]
stat_keys_dict = {"EMD":"EMD","TP":"TP","FP":"FP","Et->p":"APE","Et<-p":"ADC"}
results_df = pd.DataFrame(columns=["prediction"] + stat_keys)
for i,row in results.iterrows():
    stat_row = list()
    p = "_".join([row["dataset"], row["tool"]])
    stat_row.append(p)
    for s in stat_keys:
        stat_row.append(stat[p][stat_keys_dict[s]])
    results_df.loc[i] = stat_row

In [14]:
results_df

,prediction,EMD,TP,FP,Et->p,Et<-p
0,HIV9exp_aBayesQR,5.015615,0,10,4.555997,3.078000
1,HIV9exp_PredictHaplo,6.904978,0,2,1.475007,5.847000
2,HIV9exp_CliqueSNV,2.352555,3,2,0.223810,0.289000
3,HIV9exp_consensus,4.180000,1,0,0.000000,4.180000
4,HIV5exp_aBayesQR,14.047467,0,10,13.058573,9.400000
5,HIV5exp_PredictHaplo,9.433807,1,2,5.056757,8.400000
6,HIV5exp_CliqueSNV,7.374106,2,9,4.197593,1.400000
7,HIV5exp_consensus,14.800000,0,1,7.000000,14.800000
8,HIV7sim_aBayesQR,0.668186,5,6,0.529989,0.117188
9,HIV7sim_PredictHaplo,2.007083,3,0,0.000000,1.851562


In [15]:
datasets = experimental_datasets \
    + reduced_experimental_datasets \
    + labmix_dataset \
    + simulated_datasets
cliquesnv_stat = dict()
for d in datasets:
    for x in [0.2, 0.1, 0.05, 0.02, 0.01, 0.005, 0.002, 0.001]:
        pf = str(Path(base_dn, "results", "{}_{}p_CliqueSNV/{}.fasta".format(d, x*100, d)))
        if not os.path.exists(pf):
            continue
        rf = "relevant_haplotypes/{}.fasta".format(d.split("_")[0])
        a = !python scripts/analyze_prediction.py $pf $rf
        p = "_".join([d, str(x*100)])
        cliquesnv_stat[p] = json.loads(a[0])

In [16]:
cliquesnv_results_df = pd.DataFrame(columns=["prediction"] + stat_keys)
i=0
for d in datasets:
    for x in [0.2, 0.1, 0.05, 0.02, 0.01, 0.005, 0.002, 0.001]:
        p = "_".join([d, str(x*100)])
        if p not in cliquesnv_stat:
            continue
        stat_row = list()
        stat_row.append(p)
        a = cliquesnv_stat[p]
        for s in stat_keys:
            stat_row.append(a[stat_keys_dict[s]])
        cliquesnv_results_df.loc[i] = stat_row
        i+=1

In [17]:
cliquesnv_results_df

,prediction,EMD,TP,FP,Et->p,Et<-p
0,HIV9exp_20.0,2.528071,2,0,0.000000,2.416000
1,HIV9exp_10.0,2.429648,3,1,0.183514,0.353000
2,HIV9exp_5.0,2.429648,3,1,0.183514,0.353000
3,HIV9exp_2.0,2.352555,3,2,0.223810,0.289000
4,HIV9exp_1.0,2.541956,4,9,0.710201,0.205000
5,HIV9exp_0.5,2.485754,4,15,0.766254,0.205000
6,HIV9exp_0.2,2.593105,4,29,0.959040,0.179000
7,HIV9exp_0.1,2.589277,4,32,0.973539,0.179000
8,HIV2exp_20.0,2.289296,1,1,0.321070,0.500000
9,HIV2exp_10.0,1.707898,2,0,0.000000,0.000000


In [26]:
datasets = experimental_datasets \
    + reduced_experimental_datasets \
    + labmix_dataset \
    + simulated_datasets
cliquesnv_t_stat = dict()
for d in datasets:
    for x in [10, 15, 20, 30, 50, 70, 100, 150, 200, 300, 500]:
        pf = str(Path(base_dn, "results", "{}_2p_{}t_CliqueSNV/{}.fasta".format(d, x, d)))
        if not os.path.exists(pf):
            continue
        rf = "relevant_haplotypes/{}.fasta".format(d.split("_")[0])
        a = !python scripts/analyze_prediction.py $pf $rf
        p = "_".join([d, str(x)])
        cliquesnv_t_stat[p] = json.loads(a[0])

cliquesnv_t_results_df = pd.DataFrame(columns=["prediction"] + stat_keys)
i=0
for d in datasets:
    for x in [10, 15, 20, 30, 50, 70, 100, 150, 200, 300, 500]:
        p = "_".join([d, str(x)])
        if p not in cliquesnv_t_stat:
            continue
        stat_row = list()
        stat_row.append(p)
        a = cliquesnv_t_stat[p]
        for s in stat_keys:
            stat_row.append(a[stat_keys_dict[s]])
        cliquesnv_t_results_df.loc[i] = stat_row
        i+=1

In [31]:
pd.set_option('display.max_rows', cliquesnv_t_results_df.shape[0]+1)
print(cliquesnv_t_results_df)

               prediction       EMD TP  FP     Et->p     Et<-p
0              HIV9exp_10  2.352555  3   2  0.223810  0.289000
1              HIV9exp_15  2.352555  3   2  0.223810  0.289000
2              HIV9exp_20  2.352555  3   2  0.223810  0.289000
3              HIV9exp_30  2.352555  3   2  0.223810  0.289000
4              HIV9exp_50  2.352555  3   2  0.223810  0.289000
5              HIV9exp_70  2.352555  3   2  0.223810  0.289000
6             HIV9exp_100  2.352555  3   2  0.223810  0.289000
7             HIV9exp_150  2.352555  3   2  0.223810  0.289000
8             HIV9exp_200  2.352555  3   2  0.223810  0.289000
9             HIV9exp_300  2.352555  3   2  0.223810  0.289000
10            HIV9exp_500  2.290498  3   2  0.225263  0.289000
11             HIV2exp_10  1.872905  2   1  0.295236  0.000000
12             HIV2exp_15  1.872905  2   1  0.295236  0.000000
13             HIV2exp_20  1.872905  2   1  0.295236  0.000000
14             HIV2exp_30  1.707898  2   0  0.000000  0

### Export statistics as csv files

In [18]:
stat_dir = Path("prediction_stats")
stat_dir.mkdir(parents=True, exist_ok=True)

#### Export csv for TP and FP plot (figure 2)

In [19]:
ect_dfs = dict()
for s in stat:
    d, t = s.split("_")
    if t == "consensus":
        t = "Consensus"
    if not d in ect_dfs:
        ect_dfs[d] = pd.DataFrame(columns=["ECT","Method"])
    for e in stat[s]["ECT"]:
        ect_dfs[d] = ect_dfs[d].append({"ECT": e, "Method": t}, ignore_index=True)
for df in ect_dfs:
    ect_dfs[df].to_csv(Path(stat_dir,df+"_ECT.csv"),index=False)

In [20]:
ecp_dfs = dict()
for s in stat:
    d, t = s.split("_")
    if t == "consensus":
        t = "Consensus"
    if not d in ecp_dfs:
        ecp_dfs[d] = pd.DataFrame(columns=["ECP","Method"])
    for e in stat[s]["ECP"]:
        ecp_dfs[d] = ecp_dfs[d].append({"ECP": e, "Method": t}, ignore_index=True)
for df in ecp_dfs:
    ecp_dfs[df].to_csv(Path(stat_dir,df+"_ECP.csv"),index=False)

#### Export csv for matching distances (figure 3)

In [21]:
match_dist_df = pd.DataFrame(columns=["ADC","APE","Method","Dataset"])
for r in results_df.iterrows():
    d, t = r[1]["prediction"].split("_")
    if t == "consensus":
        t = "Consensus"
    match_dist_df=match_dist_df.append({"ADC":r[1]["Et<-p"],"APE":r[1]["Et->p"],"Method":t,"Dataset":d}, ignore_index=True)
match_dist_df.to_csv(Path(stat_dir,"match_dist.csv"),index=False)

#### Export csv for EMD (figure 4)

In [22]:
emd_df = pd.DataFrame(columns=["EMD","Method","Dataset"])
for r in results_df.iterrows():
    d, t = r[1]["prediction"].split("_")
    if t == "consensus":
        t = "Consensus"
    emd_df=emd_df.append({"EMD":r[1]["EMD"],"Method":t,"Dataset":d}, ignore_index=True)
emd_df.to_csv(Path(stat_dir,"emd.csv"),index=False)

#### Export csv for precision and recall (table 2)

In [23]:
precision_recall_df=pd.DataFrame(columns=["Dataset", "Precision", "Recall"])
for s in sorted(stat):
    precision_recall_df=precision_recall_df.append({"Dataset":s,
                                                    "Precision":stat[s]["PPV"],
                                                    "Recall":stat[s]["Sensitivity"]}, ignore_index=True)
precision_recall_df.to_csv(Path(stat_dir,"precision_recall.csv"),index=False)

In [24]:
precision_recall_df

,Dataset,Precision,Recall
0,HIV2exp_CliqueSNV,0.666667,1.000000
1,HIV2exp_PredictHaplo,0.500000,0.500000
2,HIV2exp_aBayesQR,0.111111,0.500000
3,HIV2exp_consensus,1.000000,0.500000
4,HIV5exp_CliqueSNV,0.181818,0.400000
5,HIV5exp_PredictHaplo,0.333333,0.200000
6,HIV5exp_aBayesQR,0.000000,0.000000
7,HIV5exp_consensus,0.000000,0.000000
8,HIV7sim_CliqueSNV,1.000000,0.714286
9,HIV7sim_PredictHaplo,1.000000,0.428571
